# 4-дәріс: Pandas-тың жетілдірілген әдістері

**Курс:** Машиналық оқытуға кіріспе

---

### Дәріс мақсаттары

Негізгі операцияларды меңгергеннен кейін, біз нақты деректерді талдауға және модельдер құруға дайындау үшін қажетті күрделірек және қуатты құралдарға көшеміз.

1.  **Жоқ деректермен (missing data) жұмыс істеуді үйрену:** оларды анықтау (`.isnull()`), жою (`.dropna()`) және толтыру (`.fillna()`) әдістерін меңгеру.
2.  **`GROUP BY` көмегімен деректерді агрегаттауды меңгеру:** деректерді санаттар бойынша топтастыруды және олар үшін әртүрлі статистиканы (орташа, сома, сан) есептеуді түсіну.
3.  **`.apply()` әдісін зерттеу:** деректерді күрделі түрлендіру үшін бағандарға жеке функцияларды қолдануды үйрену.
4.  **Кестелерді біріктіруді үйрену:** `DataFrames` біріктірудің екі негізгі әдісін қарастыру — қарапайым "желімдеу" үшін `pd.concat()` және SQL-дегі сияқты кілттер бойынша "ақылды" біріктіру үшін `pd.merge()`.

Бұл дағдылар Data Scientist жұмысының негізін құрайды және деректерді өңдеу бойынша тапсырмалардың басым көпшілігін шешу үшін өте маңызды.

### Кітапханаларды импорттау

In [ ]:
import pandas as pd
import numpy as np

## 1. Жоқ деректер (Missing Data)

Нақты деректер жиынында үнемі дерлік бос орындар кездеседі. Олар деректерді жинау қателерінен, толық емес жазбалардан немесе жай ғана қандай да бір ақпараттың болмауынан туындауы мүмкін. Машиналық оқыту алгоритмдерінің көпшілігі бос орындармен жұмыс істей алмайды, сондықтан оларды өңдеу қажет.

Pandas-та жоқ мәндер әдетте `NaN` (Not a Number) ретінде көрсетіледі.

**Бос орындармен жұмыс істеудің негізгі стратегиялары:**
1.  **Сол күйінде қалдыру:** Егер сіздің келесі құралыңыз (мысалы, кейбір жетілдірілген модельдер) бос орындармен жұмыс істей алса, бұл әдіс жарамды. (Артықшылығы: деректерді бұрмаламаймыз. Кемшілігі: сирек қолданылады).
2.  **Жою:** Бос орындары бар жолдарды немесе бағандарды жоюға болады. (Артықшылығы: қарапайым. Кемшілігі: деректерді жоғалтамыз, бұл өте маңызды болуы мүмкін).
3.  **Толтыру:** Бос орындарды белгілі бір мәнмен (мысалы, нөлмен, орташа, медиана немесе модамен) ауыстыру. (Артықшылығы: деректерді сақтаймыз. Кемшілігі: нәтижелерді бұрмалауы мүмкін болжамдар жасаймыз).

Демонстрация үшін шағын DataFrame құрайық.

In [ ]:
data = {'A': [1, 2, np.nan, 4],
        'B': [5, np.nan, np.nan, 8],
        'C': [9, 10, 11, 12]}
df = pd.DataFrame(data)

In [ ]:
df

### 1.1. Бос орындарды анықтау: `.isnull()`
`.isnull()` әдісі `True` мәні бос орынды білдіретін, бірдей өлшемдегі бульдік мәндері бар DataFrame қайтарады.

In [ ]:
df.isnull()

Әр бағандағы бос орындар санын тез есептеу үшін `.isnull()` нәтижесіне `.sum()` әдісін қолдануға болады.

In [ ]:
df.isnull().sum()

### 1.2. Бос орындарды жою: `.dropna()`
Ең қарапайым тәсіл — бос орындары бар жолдарды немесе бағандарды жою.

In [ ]:
# Кем дегенде бір бос орны (NaN) бар кез келген жолды жою
df.dropna()

In [ ]:
# Кем дегенде бір бос орны бар кез келген бағанды жою
df.dropna(axis=1)

In [ ]:
# thresh (шекті мән) параметрін қолдану
# Кем дегенде 2 бос емес мәні бар жолдарды ғана қалдыру
df.dropna(thresh=2)

### 1.3. Бос орындарды толтыру: `.fillna()`
Икемдірек тәсіл — бос орындарды толтыру.

In [ ]:
# Барлық бос орындарды бір мәнмен, мысалы, нөлмен толтыру
df.fillna(0)

Көбінесе бос орындарды баған бойынша орташа мәнмен толтырады. Бұл деректердің жалпы статистикасын сақтауға мүмкіндік береді.

In [ ]:
# 'A' бағанындағы бос орындарды осы бағанның орташа мәнімен толтыру
mean_A = df['A'].mean()
df['A'].fillna(mean_A)

## 2. Деректерді агрегаттау: Group By

`Group By` (топтастыру) операциясы — Pandas-тағы ең қуатты операциялардың бірі. Ол **Split-Apply-Combine** (Бөлу-Қолдану-Біріктіру) парадигмасын жүзеге асыруға мүмкіндік береді:

1.  **Split:** Деректер белгілі бір санаттық белгі негізінде топтарға бөлінеді.
2.  **Apply:** Әр топқа тәуелсіз түрде қандай да бір функция қолданылады (мысалы, `sum`, `mean`, `count`).
3.  **Combine:** Функцияны қолдану нәтижелері жаңа DataFrame-ге біріктіріледі.

Бұл "Әр санаттағы тауардың орташа бағасы қандай?" немесе "Әр қала бойынша жалпы түсім қандай?" сияқты сұрақтарға жауап беруге мүмкіндік береді.

Мысалдар үшін `mpg.csv` деректер жинағын қолданамыз.

In [ ]:
df_cars = pd.read_csv('mpg.csv')

### Топтастыру және агрегаттау функциясын қолдану

Автомобильдердің сипаттамаларының әрбір шығарылған жыл (`model_year`) үшін орташа мәндерін табайық.

In [ ]:
# 1 және 2-қадам: 'model_year' бойынша топтастырып, әр топ үшін орташа мәнді есептейміз
# Сандық емес бағандар ('name' сияқты) автоматты түрде ескерілмейді
avg_by_year = df_cars.groupby('model_year').mean(numeric_only=True)
avg_by_year.head()

Деректерді топтастырып, содан кейін агрегаттау үшін бір бағанды таңдауға болады.

In [ ]:
# Әр жыл үшін 'mpg' орташа мәні
avg_mpg_by_year = df_cars.groupby('model_year')['mpg'].mean()
avg_mpg_by_year.head()

Топтар бойынша толық статистиканы алу үшін `.describe()` сияқты басқа функцияларды да қолдануға болады.

In [ ]:
df_cars.groupby('cylinders').describe()['mpg']

## 3. `.apply()` әдісі

`.apply()` әдісі — Pandas-тың кіріктірілген функциялары жеткіліксіз болған жағдайларға арналған "швейцариялық пышағыңыз". Ол сіздің кез келген функцияңызды бағанның (Series) әрбір элементіне қолдануға мүмкіндік береді.

Автомобильдерді жанармай шығыны бойынша жіктегіміз келеді делік.

In [ ]:
# 1-қадам: Өз функциямызды құрамыз
def classify_mpg(mpg):
    if mpg < 15:
        return 'Өте төмен'
    elif 15 <= mpg < 25:
        return 'Орташа'
    else:
        return 'Жоғары'

In [ ]:
# 2-қадам: Функцияны 'mpg' бағанына қолданып, жаңа баған құрамыз
df_cars['mpg_class'] = df_cars['mpg'].apply(classify_mpg)
df_cars[['name', 'mpg', 'mpg_class']].head()

Дәл осыны **lambda-функция** көмегімен қысқаша жасауға болады:

In [ ]:
# Lambda-функциясымен мысал. Автомобильдің ауыр немесе жеңіл екенін көрсететін баған құрайық
df_cars['is_heavy'] = df_cars['weight'].apply(lambda w: 'Иә' if w > 3000 else 'Жоқ')
df_cars[['name', 'weight', 'is_heavy']].head()

## 4. DataFrame біріктіру

Сізге қажетті деректер жиі бірнеше түрлі файлдарда немесе кестелерде сақталады. Pandas оларды біріктіру үшін қуатты құралдарды ұсынады.

### 4.1. `pd.concat()`

`concat` (конкатенация) — бұл кестелерді тігінен немесе көлденеңінен қарапайым "желімдеу". Басты шарт — кестелердің құрылымы үйлесімді болуы керек.

Екі түрлі жылға арналған деректер екі файлда сақталған деп елестетейік.

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']},
                     index=[0, 1])

df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']},
                     index=[2, 3])

In [ ]:
# Тігінен біріктіру (әдепкі бойынша axis=0)
pd.concat([df1, df2])

In [ ]:
# Көлденеңінен біріктіру
pd.concat([df1, df2], axis=1)

### 4.2. `pd.merge()`

`merge` — бұл SQL-дегі `JOIN`-ға ұқсас, неғұрлым интеллектуалды біріктіру. Ол кестелерді бір немесе бірнеше ортақ бағандардағы (кілттердегі) мәндер негізінде біріктіреді.

Екі DataFrame құрайық: біреуі тіркеу туралы, екіншісі — жүйеге кіру туралы деректермен.

In [ ]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

#### Inner Merge (ішкі біріктіру)
Бұл — әдепкі біріктіру. Нәтижеге тек кілттері (`name`) **екі** кестеде де бар жолдар ғана енеді.

In [ ]:
# Pandas 'name' ортақ бағанын автоматты түрде табады
pd.merge(registrations, logins, how='inner', on='name')

#### Left Merge (сол жақты біріктіру)
Нәтижеге сол жақ кестедегі (`registrations`) **барлық** жолдар және оң жақ кестедегі тек сәйкес келетін жолдар енеді. Егер сәйкестік табылмаса, оң жақ кестедегі бағандардың мәндері `NaN`-мен толтырылады.

In [ ]:
pd.merge(registrations, logins, how='left', on='name')

## Қорытынды және келесі қадамдар

*   **Бүгін не үйрендік:**
    *   Бос орындармен жұмыс істеудің үш стратегиясы: қалдыру, жою (`.dropna()`) немесе толтыру (`.fillna()`).
    *   Деректерді агрегаттау үшін `groupby`-дағы `Split-Apply-Combine` механизмі қалай жұмыс істейтіні.
    *   `.apply()` көмегімен деректерге өз функцияларымызды қалай қолдануға болатыны.
    *   Кестелерді қалай "желімдеу" (`.concat()`) және "біріктіру" (`.merge()`) керектігі.

*   **Ары қарай не болады?**
    *   **Семинарда** біз осы әдістердің әрқайсысын практикалық мысалдармен пысықтаймыз.
    *   **Зертханалық жұмыста** сіз осы әдістерді деректер жинағын тереңірек талдау үшін қолданасыз.